# Tree of Thought approach


### Paper 
https://arxiv.org/pdf/2305.10601.pdf

This notebook demonstrates the use of Tree of thought as a 4 step process for a creative writing task.

1. Create *K* plans to approach the task
2. Vote the best 1 out of the *K* plans
3. Create *N* passages with the best plan
4. Vote the best 1 out of the *N* passages

### Task

In this demonstration, we will use LLM to create a product marketing slogan. The marketing lead has some thoughts in mind that she would like the LLM to follow. Refer to the standard prompt that describes the marketing lead's preferences.


### Note:
1. Since we are using commerical API, it will cost you $ to run the code
2. The code can be adjusted for similar tasks by tweaking the prompts, rest of the code stays the same

### Langchain
Use of Langchain will simplify the code below. Langchain has not been used deliberately to show you the concept of LLM chains :) in the next iteration you will use Langchain to rewrite this code. If you already know Langchain, then feel free to use it instead of the solution proposed in this notebook.

## Setup the environment variable for API access

#### NOTE
You MUST change the path to your own environment file

In [56]:
from dotenv import load_dotenv
import os

import warnings

warnings.filterwarnings("ignore")

# Load the file that contains the API keys
load_dotenv('C:\\Users\\raj\\.jupyter\\.env')

True

### Use OpenAI to run infrence

https://platform.openai.com/docs/api-reference/chat

In [57]:
from openai import OpenAI

client = OpenAI()

# utility function for running infrence
def  run_openai_infrence(prompt, temperature=0.5, max_tokens=400, stop_sequence=[], count=1):
    client = OpenAI()
    
    response_content = []

    for i in range(count):
        response = client.chat.completions.create(
          model="gpt-3.5-turbo",
          messages=[
            {
                "role": "system", 
                "content": "you are an expert in generating product marketing material"
            },
            {
              "role": "user",
              "content": prompt
            }
          ],
          temperature=temperature,
          max_tokens=max_tokens,
          stop=stop_sequence
        )

        response_content.append(response.choices[0].message.content)
        
    return response_content

## Response without ToT

We will first create a prompt and get a response from Open AI/GPT.

In [ ]:
# These are output indicators  used in the prompt
ouput_passage_indicator = "Passage:"
ouput_plan_indicator = "Plan:"
output_best_plan_indicator = "Best Plan:"
output_best_passage_indicator = "Best Passage:"

In [58]:
# This is a standard prompt
prompt_instructions = """Write a passage to describe our new product:
Javelin 20 High Performance Shoes.

The passage should be no more than 100 words.

The title must contain the sentence: 
A new you

The first sentence must contain a few of these words: 
motivation, workout, sports, attractive

The last few sentences must motivate the reader to visit out website http://acme.com/jav20

The passage must contain the phrase : 
trend setter
"""

prompt = prompt_instructions + ouput_passage_indicator + "\nYour passage here."
# print( prompt )

In [59]:
# response to the standard prompt
response = run_openai_infrence(prompt)

print(response[0])

A new you awaits with our Javelin 20 High Performance Shoes. Designed to ignite your motivation during workouts, these sports shoes are not only attractive but also engineered to enhance your performance. Whether you're a trend setter or a fitness enthusiast, our Javelin 20 shoes will elevate your game to new heights. Experience the perfect blend of style and functionality, allowing you to conquer any challenge with confidence. Don't miss out on this opportunity to revolutionize your fitness journey. Visit our website at http://acme.com/jav20 and be prepared to unleash your true potential.


## Response with ToT

### 1. Generate 3 plans

In [60]:
prompt = prompt_instructions + """
Your output will be in following format.

{ouput_plan_indicator}
Your plan here.

{ouput_passage_indicator}
Your passage here.
""" 

prompt = prompt.format(ouput_plan_indicator=ouput_plan_indicator, ouput_passage_indicator=ouput_passage_indicator)


print(prompt)

Write a passage to describe our new product:
Javelin 20 High Performance Shoes.

The passage should be no more than 100 words.

The title must contain the sentence: 
A new you

The first sentence must contain a few of these words: 
motivation, workout, sports, attractive

The last few sentences must motivate the reader to visit out website http://acme.com/jav20

The passage must contain the phrase : 
trend setter

Your output will be in following format.

Plan:
Your plan here.

Passage:
Your passage here.



In [61]:
# use stop sequence to generate just the plan
passage_stop_sequence = [ouput_passage_indicator]

# This will return an array with 3 elements
plans_for_passage = run_openai_infrence(prompt,0.8,400,passage_stop_sequence,3)

In [62]:
for plan in plans_for_passage:
    print(plan)

Plan:
Title: A new you
First sentence: Get motivated for your workout with our attractive Javelin 20 high performance shoes designed for sports enthusiasts.
Last few sentences: Discover the trendsetter in you by visiting our website at http://acme.com/jav20 and get ready to experience the ultimate in comfort and style.


Plan:
- Introduce the product as Javelin 20 High Performance Shoes
- Mention its features and benefits
- Highlight its appeal for sports enthusiasts and fitness enthusiasts
- Emphasize the attractive design and performance capabilities
- Provide a call to action to visit the website for more information


Plan:
Title: A new you
First sentence: Motivation, sports, workout, attractive
Last sentences: Motivate reader to visit website http://acme.com/jav20
Phrase: Trend setter




### 2. Ask LLM to vote on the plan

In [63]:
prompt_plan_vote = """
3 experts were given the following task.

Task:

{prompt_instructions}

you need to review the 3 plans and identify the best plan out of the 3. 

Output your response as follows, where N is the plan number. 
{output_best_plan_indicator} N

Give 3 reasons why you select the plan.

{plans}

"""

prompt_plans = ""
for i, plan in enumerate(plans_for_passage):
    plan = plan.replace(ouput_plan_indicator, '')
    prompt_plan = "Plan {i}:{plan}".format(i=(i+1), plan=plan)
    
    prompt_plans = prompt_plans + prompt_plan

prompt = prompt_plan_vote.format(prompt_instructions=prompt_instructions, plans = prompt_plans, output_best_plan_indicator=output_best_plan_indicator)
print(prompt)


3 experts were given the following task.

Task:

Write a passage to describe our new product:
Javelin 20 High Performance Shoes.

The passage should be no more than 100 words.

The title must contain the sentence: 
A new you

The first sentence must contain a few of these words: 
motivation, workout, sports, attractive

The last few sentences must motivate the reader to visit out website http://acme.com/jav20

The passage must contain the phrase : 
trend setter


you need to review the 3 plans and identify the best plan out of the 3. 

Output your response as follows, where N is the plan number. 
Best Plan: N

Give 3 reasons why you select the plan.

Plan 1:
Title: A new you
First sentence: Get motivated for your workout with our attractive Javelin 20 high performance shoes designed for sports enthusiasts.
Last few sentences: Discover the trendsetter in you by visiting our website at http://acme.com/jav20 and get ready to experience the ultimate in comfort and style.

Plan 2:
- Introd

In [64]:
# use stop sequence to generate just the plan
passage_stop_sequence = [ouput_passage_indicator]

# This will return an array with 3 elements
plan_voted_best = run_openai_infrence(prompt,0.8,400)

In [65]:
# Get the index of the best plan by searching for the "Best Plan:"
best_plan_index = plan_voted_best[0].find(output_best_plan_indicator)
temp_index = best_plan_index+len(output_best_plan_indicator)
new_line_index  = plan_voted_best[0].find("\n",temp_index)
best_plan_index = int(plan_voted_best[0][temp_index : new_line_index]) - 1

print('Best Plan Index:', best_plan_index)
print('------')
print(plan_voted_best[0])

Best Plan Index: 1
------
Best Plan: 2

Reasons for selecting Plan 2:

1. Plan 2 provides a clear and concise introduction of the product, Javelin 20 High Performance Shoes, while also highlighting its features and benefits. This helps potential customers understand the value proposition of the product right from the start.

2. The plan effectively emphasizes the appeal of the product to sports enthusiasts and fitness enthusiasts, which is likely the target audience for high-performance shoes. By addressing the specific needs and interests of this audience, the plan increases the chances of attracting the right customers.

3. Plan 2 includes a call to action to visit the website for more information, which is crucial for driving traffic and conversions. By directing readers to the website, the plan enables them to explore further details about the product, ultimately increasing the likelihood of making a purchase.

In summary, Plan 2 effectively introduces the product, highlights its a

## 3. Prompt LLM to generate 3 passages using the best plan

In [66]:
prompt_generate_3_passages = """
Create three passages using the following instructions and plan.

Instructions:
{prompt_instructions}

{best_plan}

Your output will be in following format.

{ouput_passage_indicator}
Your response here


"""

# Use the plans generated in step-1, best plan index from step 2
best_plan = plans_for_passage[best_plan_index]

prompt = prompt_generate_3_passages.format(prompt_instructions=prompt_instructions, 
                                           best_plan=best_plan,
                                           ouput_passage_indicator=ouput_passage_indicator)


In [67]:
print(prompt)


Create three passages using the following instructions and plan.

Instructions:
Write a passage to describe our new product:
Javelin 20 High Performance Shoes.

The passage should be no more than 100 words.

The title must contain the sentence: 
A new you

The first sentence must contain a few of these words: 
motivation, workout, sports, attractive

The last few sentences must motivate the reader to visit out website http://acme.com/jav20

The passage must contain the phrase : 
trend setter


Plan:
- Introduce the product as Javelin 20 High Performance Shoes
- Mention its features and benefits
- Highlight its appeal for sports enthusiasts and fitness enthusiasts
- Emphasize the attractive design and performance capabilities
- Provide a call to action to visit the website for more information



Your output will be in following format.

Passage:
Your response here





In [68]:
# This will return an array with 3 elements
passages_based_on_best_plan = run_openai_infrence(prompt,temperature=0.5,max_tokens=400,stop_sequence=[],count=3)

In [69]:
for passage in passages_based_on_best_plan:
    print(passage)

A New You: Javelin 20 High Performance Shoes are the perfect motivation for your workout routine. Designed for sports and fitness enthusiasts, these shoes are a trendsetter in the industry. With their sleek and attractive design, they not only make a fashion statement but also enhance your performance. The Javelin 20 High Performance Shoes offer superior comfort, durability, and support, allowing you to push your limits and achieve your fitness goals. Don't miss out on this opportunity to elevate your workouts. Visit our website http://acme.com/jav20 to learn more about these game-changing shoes and unleash your full potential.
Title: A New You with Javelin 20 High Performance Shoes

Passage:
Looking for some motivation to enhance your workout routine? Look no further than our latest innovation, Javelin 20 High Performance Shoes. Designed for trend setters like you, these shoes are the perfect companion for your sports activities. With their exceptional features and benefits, Javelin 2

### 4. Vote the best out of the 3 passages

In [70]:
prompt_passage_vote = """
3 experts were given the following task.

Task:
{prompt_instructions}

you need to review the 3 passages and identify the best passage out of the 3. 

Output your response as follows, where N is the passage number. 
{output_best_passage_indicator} N

Give 3 reasons why you select the passage.

{passages}

"""

prompt_passages = ""
for i, passage in enumerate(passages_based_on_best_plan):
    passage = passage.replace(ouput_passage_indicator, '')
    prompt_passage = "Passage {i}:{passage}\n\n".format(i=(i+1), passage=passage)
    
    prompt_passages = prompt_passages + prompt_passage

prompt = prompt_passage_vote.format(prompt_instructions=prompt_instructions, 
                                    passages = prompt_passages, 
                                    output_best_passage_indicator=output_best_passage_indicator)
print(prompt)


3 experts were given the following task.

Task:
Write a passage to describe our new product:
Javelin 20 High Performance Shoes.

The passage should be no more than 100 words.

The title must contain the sentence: 
A new you

The first sentence must contain a few of these words: 
motivation, workout, sports, attractive

The last few sentences must motivate the reader to visit out website http://acme.com/jav20

The passage must contain the phrase : 
trend setter


you need to review the 3 passages and identify the best passage out of the 3. 

Output your response as follows, where N is the passage number. 
Best Passage: N

Give 3 reasons why you select the passage.

Passage 1:A New You: Javelin 20 High Performance Shoes are the perfect motivation for your workout routine. Designed for sports and fitness enthusiasts, these shoes are a trendsetter in the industry. With their sleek and attractive design, they not only make a fashion statement but also enhance your performance. The Javelin 

In [71]:
# use stop sequence to generate just the plan
passage_stop_sequence = [ouput_passage_indicator]

# This will return an array with 3 elements
passage_voted_best = run_openai_infrence(prompt,0.8,400)

In [72]:
# Get the index of the best pasage by searching for the "Best Passage:"
best_passage_index = passage_voted_best[0].find(output_best_passage_indicator)
temp_index = best_passage_index+len(output_best_passage_indicator)
new_line_index  = passage_voted_best[0].find("\n",temp_index)
best_passage_index = int(passage_voted_best[0][temp_index : new_line_index]) - 1

print('Best Passage Index:', best_passage_index)
print('------')
print(passage_voted_best[0])

Best Passage Index: 0
------
Best Passage: 1

Reasons for selection:
1. The passage effectively uses the title "A New You" in the opening sentence, capturing the attention of the reader and conveying the idea of transformation.
2. The passage incorporates the words "motivation," "workout," "sports," and "attractive" in the first sentence, effectively conveying the key features of the product.
3. The passage successfully motivates the reader to visit the website http://acme.com/jav20 by emphasizing the unique qualities of the Javelin 20 High Performance Shoes and the opportunity to enhance their workouts and unleash their full potential.


In [73]:
print("Final Result")
print("------------")
print(passages_based_on_best_plan[best_passage_index])

Final Result
------------
A New You: Javelin 20 High Performance Shoes are the perfect motivation for your workout routine. Designed for sports and fitness enthusiasts, these shoes are a trendsetter in the industry. With their sleek and attractive design, they not only make a fashion statement but also enhance your performance. The Javelin 20 High Performance Shoes offer superior comfort, durability, and support, allowing you to push your limits and achieve your fitness goals. Don't miss out on this opportunity to elevate your workouts. Visit our website http://acme.com/jav20 to learn more about these game-changing shoes and unleash your full potential.
